# SIXT33N Project
## Phase 6: Advanced Controls (and more!)

### EE 16B: Designing Information Devices and Systems II, Fall 2018

Written by Vivek Athalye and Andrew Blatner (2017)

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

## Table of Contents

* [Introduction](#intro)
* [Task 1: Launchpad Implementation of PCA Classify](#task1)
* [Task 2: Analyzing a Turn](#task2)
* [Task 3: Implementing Turns](#task3)
* [Task 4: Correcting Mechanical Errors](#task4)

<a id='intro'></a>
## Introduction

This week, we will first implement port our PCA classifier from the iPython notebook to a file that can be run on the Launchpad. This will allow us to integrate PCA with the rest of the project.

Then, we will revisit controls. Recall that our control scheme for SIXT33N modeled each wheel in the following way:

$$d_R[k+1] = d_R[k] + \theta_R u_R[k] - \beta_R$$

The open loop model has an input $u[k]$, corresponding to the PWM level, and outputs the distance for that wheel. We implemented a closed-loop controller by using the desired velocity and the difference between the wheel distances, $$\delta[k] = d_L[k] - d_R[k]$$.

In this phase, we will implement turns by modifying the closed loop control scheme we developed to make SIXT33N move straight.

<a id='task1'></a>
## <span style="color:blue">Task 1: Launchpad Implementation of PCA Classify</span>


### Materials
- Microphone front-end circuit
- Launchpad + USB


**<span style="color:red">IMPORTANT: remove the 5V jumper (pictured below) for this step. It can cause interference during classification.</span>**

<img width=400px src="images/5vjumper.jpg">

This section will walk you through implementing your classification algorithm on the Launchpad. In addition, you will need to transfer the PCA vectors and mean to the Launchpad. **Use the code below to format vectors for the Launchpad code.**

In [ ]:
a = ''
v = 
for i in v[0,:].T:
    a += str(i) + ', '
print("PC1")
print(a)
print(" ")


Your last step will be to implement your <b>data processing</b> and <b>classification</b> (just the projection, not the PCA) in the Launchpad sketch <b>`classify.ino`</b>. Since Energia does not have as many in-built functions as Python, you might have to write out the functions yourself. For example, a dot product should be written as:

`float result = 0;`<br/>
`for (i=0; i<LENGTH; i++){`<br/>
&emsp; `result += vector1[i]*vector2[i];`<br/>
`}`

For debugging purposes, printing to Energia's serial monitor looks like the line below.

<code>Serial.println("I'm being printed!");</code>

There are 3 code blocks (`PCA1/2/3`) that you need to modify. <b>You should not have to change anything else outside these marked code blocks.</b> 

**`CODE BLOCK PCA1`**
- Copy `SNIPPET_SIZE`, `PRELENGTH` and `THRESHOLD` from this IPython Notebook.
- Read the following to set `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD`.

**`KMEANS_THRESHOLD` filters the classification depending on a sample's distance from the closest centroid.** If the L2 norm (distance) is larger than the threshold, your classification algorithm should simply ignore it and wait for the next sample. Look at the plot of sample data and the centroids from the PCA Notebook and approximate a radius around the centroids that capture most of the data. **Try to be conservative - it's better to not classify than to misclassify.**

**`LOUDNESS_THRESHOLD` filters the classification depending on the amplitude of the recorded data.** If the recorded data is too soft, we do not want to classify it as it is probably noise. Since the loudness unit is arbitrary, start by using `700`. Later, if the Launchpad classifies noise, increase this constant. If it misses a lot of speech (i.e. thinks your word is noise), decrease this constant. This variable is used internally in the enveloping function.

**`CODE BLOCK PCA2`**
- Copy the PCA vectors, mean, and centroids from this IPython notebook.
- If you need more than 2 principal components, add a new `pca_vec3` array. 
- Using more principal components increases the dimensionality of the centroids and projections.

**`CODE BLOCK PCA3`**
- This is the actual classification algorithm.
- Before this block, the call to `envelope` leaves the data vector in the array called `result`.
- Project this data onto your new PCA basis.
    - Remember to subtract the mean vector, before doing a dot product for each basis.
    - Use the variables `proj1` and `proj2` to store the projection results.
- Classify the projections using the centroids.
    - Find the distance between the projected data point and each centroid using the function `l2_norm` (for 2 principal components) or `l2_norm3` (for 3 principal components). Look up the function definition in the sketch.
    - Out of the 4 centroids, find the one with the smallest L2 norm.
    - Verify this distance is less than `KMEANS_THRESHOLD`.
- Print the classification to the serial monitor.

Before testing the code, probe the circuit's output with the oscilloscope and make sure that it still averages around 1.65V. Now upload the sketch, (re)open the serial monitor, and press the reset button. Say your word and the Launchpad should recognize it!

**<span style="color:red">If the Launchpad does not classify as well as you think it should, remember to play with the `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD` variables.</span>** To debug the sketch, you can also print out any of the variables you have used. 

Voila! Your SIXT33N can recognize your words!

**<span style="color:red">Summary TO DO</span>** 
- **<span style="color:red">Fill in `CODE BLOCK PCA1`: Fill out `SNIPPET_SIZE`, `PRELENGTH`, `THRESHOLD`, `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD`</span>** 
- **<span style="color:red">Fill in `CODE BLOCK PCA2`: Copy the principal components, mean vector and centroids from the IPython notebook</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA3`: Do the actual classification. </span>**

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">
<br/>
## <span style="color:green">CHECKPOINT 1</span>
<span style="color:green">**Show your GSI the Launchpad recognizing words.** Make sure the correct identified word is printed in the Serial Monitor.</span>

<a id='task2'></a>
## <span style="color:blue">Task 2: Analyzing a Turn</span>

### Materials
- SIXT33N Car
- Closed-loop control scheme

**<span style="color:red">IMPORTANT: replace the 5V jumper (pictured below) for the rest of the lab.</span>**

<img width=400px src="images/5vjumper.jpg">

First, verify that your closed loop control scheme is still functional. Load `closed_loop.ino` onto your launchpad and make sure your car drives straight.

### We want to turn with a specific radius.  What open-loop control input would lead to turning?  
During open-loop control, we sought to drive the car straight at our desired velocity $v^*$.  

We used our model of the car ($\theta$, $\beta$) to calculate the open-loop control input which drives the car straight in the absence of perturbations:  

$$u^{OL}_L = \frac{v^*+\beta_L}{\theta} $$
$$u^{OL}_R = \frac{v^*+\beta_R}{\theta} $$

Is it possible to calculate an open-loop control input which would result in turning with a desired radius?


### We have been turning during Closed-loop Control
We can get a clue about turning by considering how we did closed-loop control. We perform feedback control of SIXT33N in order to minimize the difference between the two wheels ($\delta[k] = d_L[k] - d_R[k]$). When perturbations cause one wheel to get ahead of the other resulting in non-zero $\delta$, feedback control turns the car to correct the error.  

Let's walk through this.  

Our feedback control policy is: 
$$u_L[k] = u_L^{OL} + \frac{k_L}{\theta_L}\delta[k]$$

$$u_R[k] = u_R^{OL} + \frac{k_R}{\theta_R}\delta[k]$$


Let's say the right wheel has moved further than the left ($d_R[k] > d_L[k]$), resulting in a negative $\delta[k]$. A negative $\delta[k]$ results in a reduction of $u_R[k]$, and an increase of $u_L[k]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So $\delta \ne 0$ can be used to turn the car. Let's get more precise.  


### Turning via reference tracking

We would like the car to turn with a specified radius $r$ and speed $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of wheel circumference.

To turn, we want $\delta$ to change at a particular rate. Without loss of generality, we'll analyze a right turn, corresponding to increasing $\delta$. For a left turn, we simply negate $\delta$. Our goal is to generate a reference from the desired $r$ and $v^*$ for the controller to follow. This reference will be a function of the controller's time-step.

Use the following variables:
- $k$ - time-step
- $r$ [cm] - turn radius of the center of the car; 1 cm ~ 1 encoder tick
- $d$ [ticks] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm ~ 1 encoder tick
- $\omega$ [rad/tick] - angular velocity
- $\theta$ [rad] - angle traveled

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>



From this geometry, can you write $\delta[k]$ in the following form?
$$\delta[k] = f(r,v^*,l,k)$$

<h3><span style='color:red'>**Check your answer with the lab staff!**</span></h3>


<a id='task3'></a>
## <span style="color:blue">Task 3: Implementing Turns</span>

### Materials
- SIXT33N Car
- Closed-loop control scheme
- Plan from Task 1

### Sampling periods 
 
In our implementation, the control loop and the data collection have different periods, $T_c$ and $T_d$ respectively.  $T_d$ is an integer multiple of $T_c$: $m=\frac{T_d}{T_c} \in \mathbb{Z}^{+}$
 
The units of $v^*$ are [ticks/$T_d$] and the units of $k$ are [seconds/$T_c$]
 
We use $T_c=100$ms, $T_d=500ms$, so $m=5$.

**<span style='color:red'> Replace $v^*$ with $\frac{v^*}{m}$ [ticks/$T_c$]</span>**

### Implementation
- Open `turning.ino`.
- Copy the required code from `closed_loop.ino` into **`CODE BLOCK CON1/2/3`**
    - Make sure to update pin numbers if you've made changes.
    - Setting `delta_ss` helps your car's driving converge to straight sooner!
- **Complete the function `delta_reference` in `CODE BLOCK CON4`**.
    - Look at `CODE BLOCK CON0` to see how this is used!
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR WIDTH`, and `k`.
    - `delta_ss` is used in `CODE BLOCK CON0`, so don't include it here.
- Change the array of run times so the car goes straight for a longer distance, left, straight for a shorter distance, and right for the duration you want.

When you're done, try it out! SIXT33N should go straight, turn left, go straight, and turn right with a stop between each stage. It's ok if SIXT33N does not turn 90 degrees - as long as it turns you should be fine. Just make sure that it is clear when SIXT33N is turning, and when it is driving straight.

<a id='task4'></a>
## <span style="color:blue">Task 4: Correcting Mechanical Errors</span>

At this stage, your car may already go perfectly straight and turn left and right equally. However, some cars may not go quite straight even when both encoder measurements are exactly the same. This can be caused by mechanical issues such as axle wobble or mismatch in the wheel sizes. In this case, the car thinks it's driving straight, so it's the best we can do using just the sensors in our control system.

If the car is consistently turning slightly instead of driving straight, we can correct it by instructing the car to slightly turn in the opposite direction. 

**If you think your car could benefit from this, implement the function `straight_correction` in `CODE BLOCK CON5` using the variable `STRAIGHT_RADIUS`.**

<span style='color:red'>**Show your GSI the controlled SIXT33N!**</span>

In the final phase of the project, you will integrate all of the parts and control SIXT33N with your voice.

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">
<br/>
## <span style="color:green">CHECKPOINT 2</span>
 <span style="color:green">**Show your GSI how SIXT33N moves using turning.ino.** </span>